# BioImagePy: runner or process execution

This tutorial descibe the use of the **Runner** class to run a process. A process runable with the **Runner** class is a command line tool descibed by a *XML* file. The structure and content of a process *XML* file is not detailled in this tutorial. Please refer to the **BioImageIT** project documentation or to the **Galaxy Project** wrapper documentation to know more about *XML* process file or wrappers.

In this tutorial, we assume that we have the process `sdeconv2d` available. What we call a process here is a executable program and it associated *XML* wrapper.


## Setup the process

In [1]:
import sys

# this is an optional command to link BioImagePy from local directory
sys.path.append("../bioimagepy") # Change this path to the bioimagepy directory

# initialize the configuration
from bioimagepy.config import ConfigAccess
ConfigAccess("../config.json")

To run a process you need first to instantiate a **Process**. A **Process** is a class than contains all the matadata of the process. 
As it is tedious to instantiate a **Process** with the path of the XML file, **BioImagePy** provides a **ProcessAccess** class that manage an *XML wrappers* database. A process can than be instantiate by it fullname (ie name plus version)

In [5]:
from bioimagepy.process import ProcessAccess, Process

# show the list of all available processes
processAccess = ProcessAccess()
processAccess.search()

sampletool_v0.1.0	     SampleTool	          0.1.0	     sequential	/Users/sprigent/Documents/code/bioimage-it/package/bioimageit/toolboxes/tools/sampletool/tools/sampletool.xml
threshold_particles_v1.0.0	Count particles	          1.0.0	     sequential	/Users/sprigent/Documents/code/bioimage-it/package/bioimageit/toolboxes/tools/fiji/tools/threshold_particles.xml
Wilcoxon_v1.0.0	       Wilcoxon	          1.0.0	          merge	/Users/sprigent/Documents/code/bioimage-it/package/bioimageit/toolboxes/tools/htesting/tools/wilcoxon.xml
svdeconv2d_v0.1.0	    SPARTION 2D	          0.1.0	     sequential	/Users/sprigent/Documents/code/bioimage-it/package/bioimageit/toolboxes/tools/simg/tools/svdeconv2d.xml
svdeconv3d_v0.1.0	    SPARTION 3D	          0.1.0	     sequential	/Users/sprigent/Documents/code/bioimage-it/package/bioimageit/toolboxes/tools/simg/tools/svdeconv3d.xml


In [6]:
# get the svdeconv2d process
process = ProcessAccess().get('sdeconv2d_v0.1.0')
if process:
    process.man()
else:
    print("Process not found")    

SPARTION 2D : 
	              i	               	    Input Image
	          sigma	              3	          Sigma
	 regularization	              7	 Regularization
	      weighting	            0.6	      Weighting
	         method	            HSV	         Method
	              o	               	 Denoised image


## Setup the runner

Now we can instantiate the runner with the process

In [7]:
from bioimagepy.runner import Runner

runner = Runner(process)
runner.man()

SPARTION 2D : 
	              i	               	    Input Image
	          sigma	              3	          Sigma
	 regularization	              7	 Regularization
	      weighting	            0.6	      Weighting
	         method	            HSV	         Method
	              o	               	 Denoised image


The `man()` method prints a man page that describe the process. Thus we need to execute the `svdeconv` process with four parameters.

## Execute on files with exec()

To execute the process we simply run the method exec with the parameters descibed in the `man()` method of the process:

In [10]:
import imageio

runner.exec('i', 'synthetic_data/data/population1_001.tif', 
            'o', 'population1_001_deconv.tif', 
            'sigma', 2,
            'regularization', 2, 
            'weighting', 0.1,
            'method', 'SV') 

myoutput = imageio.imread('population1_001_deconv.tif')

run singularity container: docker://registry.gitlab.inria.fr/serpico/simg:2ef8ff9b5495a722ef03188106e912371dc9d46d
args: ['svdeconv2d', '-i', 'synthetic_data/data/population1_001.tif', '-o', 'population1_001_deconv.tif', '-sigma', '2', '-regularization', '2', '-weighting', '0.1', '-method', 'SV', '-niter', '200']


FileNotFoundError: No such file: '/Users/sprigent/Documents/code/bioimage-it/package/bioimageit/bioimagepy-tutorial/population1_001_deconv.tif'

Lets visualize the output:

In [ ]:
import matplotlib.pyplot as plt

plt.figure(1)                      
plt.imshow(imageio.imread('synthetic_data/data/population1_001.tif'))
plt.figure(2)
plt.imshow(myoutput)
plt.show()

This `exec()` method is usefull when we need to run a process on a list of data stored in a file system. Because, in this case, the data are not loaded into python variable but sent directly to the external process. 

## Execute on python data with run()

The `run()` method allows to run the process direcly on python data. It usage is similar to `exec()` except that the ouptut data are returned.

In [ ]:
import matplotlib.pyplot as plt

# load an image
input_image = imageio.imread('synthetic_data/data/population1_001.tif')

# run the process
output_image = runner.run('i', input_image,
               'regularization', 1, 
               'weighting', 0.1,
               'method', 'SV') 

# visualize the images
plt.figure(1)                      
plt.imshow(input_image)
plt.figure(2)
plt.imshow(output_image)
plt.show()

The run process prints the executed command. As we can see, the input the output images are saved in the temporary directory to send them to the process. The advantage of the `run()` method is that we do not need to write code to send the data to the process (files saving and loading) and can just send the data to the process as if it where a native python function.

## Wrapping up

In this short tutorial, we saw the usage of the **Runner** and **Process** classes. Basically we need to know 3 methods
1. `man`: to get the list of the process I/O and parameters
2. `exec`: to execute the process on files
3. `run`: to execute the process on data stored in python variables

This **Runner** class is a single data level processing and does not generate any metadata. To run a process on an experiment dataset and have automatically generated metadata, we use the **Pipeline** class in the next tutorial